In [ ]:
!pip install -U openai-whisper
!pip install srt
!pip install googletrans==4.0.0rc1

In [ ]:
import subprocess
import whisper
from datetime import timedelta
import srt
from srt import Subtitle


# 音声データ抽出 (音声ファイル名変更)
subprocess.run(['ffmpeg', '-i', 'test.mp4', '-acodec', 'libmp3lame', '-ab', '256k', 'audio.mp3'])

# Whisperモデル読み込み
model = whisper.load_model("medium")

# 音声ファイル読み込み
audio = whisper.load_audio("audio.mp3")

# 音声認識 (開始・終了時間付きセグメント取得)
result = model.transcribe(audio, verbose=True)
segments = result["segments"]

# SRTファイル用の空リスト初期化
subtitles = []

# セグメントごとにSRT行を作成
for index, segment in enumerate(segments, start=1):
    start_time = timedelta(seconds=segment["start"])
    end_time = timedelta(seconds=segment["end"])
    text = segment["text"]

    subtitle = Subtitle(
        index=index,
        start=start_time,
        end=end_time,
        content=text
    )
    subtitles.append(subtitle)

# SRTファイル内容の作成
srt_content = srt.compose(subtitles)

# ファイルへの書き込み (エンコーディング指定)
with open("output.srt", "w", encoding="utf-8") as f:
    f.write(srt_content)

# SRTファイル読み込み (エンコーディング指定)
with open('output.srt', 'r', encoding='utf-8') as file:
    srt_data = file.read()

In [ ]:
import logging
import time
from googletrans import Translator
from requests.exceptions import ReadTimeout
import srt
from srt import Subtitle

# Initialize logger
logging.basicConfig(level=logging.INFO)

# Initialize translator
translator = Translator()

def safe_translate(text, src='en', dest='ja', max_attempts=5):
    for attempt in range(max_attempts):
        try:
            return translator.translate(text, src=src, dest=dest)
        except ReadTimeout:
            logging.warning(f"Attempt {attempt+1}/{max_attempts}: ReadTimeout encountered. Retrying...")
            if attempt < max_attempts -  1:
                time.sleep((attempt +  1) *  0.5)
            else:
                raise Exception(f"Translation failed after {max_attempts} attempts.")

def read_srt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def write_srt_file(file_path, content):
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(content)

def translate_subtitles(srt_data):
    subtitles = list(srt.parse(srt_data))
    for subtitle in subtitles:
        original_text = subtitle.content
        translated_text = safe_translate(original_text).text
        subtitle.content = translated_text
        logging.info(f"Translated: {original_text} -> {translated_text}")
        time.sleep(0.1)
    return subtitles

def translate_srt(input_file, output_file):
    srt_data = read_srt_file(input_file)
    translated_subtitles = translate_subtitles(srt_data)
    translated_srt_content = srt.compose(translated_subtitles)
    write_srt_file(output_file, translated_srt_content)

# Example usage
translate_srt('output.srt', 'translated_file.srt')



In [ ]:
# 改行関数 (句読点・長さに応じて改行)
def add_line_breaks(text, max_length=30, max_lines=3):
    result = ""
    current_line = ""
    lines = []

    for char in text:
        if char == "、" or char == "。":
            current_line += char
            if len(current_line) > max_length or char == "。":
                lines.append(current_line)
                current_line = ""
        else:
            current_line += char

    if current_line:
        lines.append(current_line)

    # 行数を制限する
    while len(lines) > max_lines:
        line1 = lines.pop(0)
        line2 = lines.pop(0)
        combined_line = line1 + line2
        lines.insert(0, combined_line)

    return "\n".join(lines)

# 改行関数 (SRT形式保持)
def add_line_breaks_srt(srt_text):
    lines = srt_text.split("\n")
    result = ""

    for i, line in enumerate(lines):
        if line.strip().isdigit():
            result += line
        elif "-->" in line:
            result += line
        else:
            result += add_line_breaks(line) + "\n"

        # 空行を追加 (字幕間の空白)
        if i < len(lines) - 1 and lines[i + 1] != "" and line != "":
            result += "\n"

    return result.strip()

# 改行されたテキストを取得
with open('translated_file.srt', 'r', encoding='utf-8') as file:
    srt_text = file.read()

# 改行されたテキストを取得
result = add_line_breaks_srt(srt_text)

# ファイルへの書き込み (エンコーディング指定)
with open("translated_file.srt", "w", encoding='utf-8') as file:
    file.write(result)

In [ ]:
import subprocess

subprocess.run([
    'ffmpeg',
    '-hwaccel', 'auto',  # ハードウェアアクセラレーションを有効にする
    '-i', 'test.mp4',
    '-vf', "subtitles=translated_file.srt:force_style='FontName=Helvetica,FontSize=12'",
    '-c:v', 'h264_amf',  # AMDのハードウェアエンコーダーを利用
    '-c:a', 'copy',  # オーディオストリームをコピーする
    'translated_subtitle.mp4'
])